In [1]:
import time
from tqdm import tqdm

In [2]:
import tensorflow as tf

2024-05-01 12:37:34.352758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 12:37:34.927881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from tensorflow.keras.models import load_model

In [4]:
import numpy as np

In [5]:
model = load_model('fullfreq2-7_surogate.h5')

2024-05-01 12:37:36.500802: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-05-01 12:37:36.501269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 578 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [38]:
def dgn_obj(s_para, desired_dB, outfield_dB, desired_penalty_rate, outfield_penalty_rate):
    s_para = tf.reshape(s_para, (num_nodes, -1, 2))
    E = tf.reduce_sum(tf.square(s_para), axis=-1)
    
    desired_E = E[: , d_lb: d_rb+1]

    desired_P = 10**(-desired_dB / 10)
    desired_condition = desired_E < desired_P
    a = desired_P**(1 - desired_penalty_rate)
    # desired_E = tf.where(desired_condition, a * tf.pow(desired_E, desired_penalty_rate), desired_E)
    desired_E = tf.where(desired_condition, desired_P - desired_E, 0)

    desired = tf.reduce_sum(desired_E, axis=-1)

    outfield_E = tf.concat([E[: , : o_l + 1], E[: , o_r: ]], axis=-1)


    outfield_P = 10**(-outfield_dB / 10)
    outfield_condition = outfield_E > outfield_P
    b = outfield_P**(1 - outfield_penalty_rate)
    # outfield_E = tf.where(outfield_condition, b * tf.pow(outfield_E, outfield_penalty_rate), outfield_E)
    outfield_E = tf.where(outfield_condition, outfield_E - outfield_P, outfield_E)
    
    outfield = tf.reduce_sum(outfield_E, axis=-1)
    
    return 3 * desired + outfield

In [39]:
# 参数数量，迭代次数
num_nodes = 5000
num_node_epochs = 1500

In [40]:
# 物理边界
mmin = np.array([200, 200, 200, 200, 200, 200, 200, 9000, 100])
mmax = np.array([2000, 2000, 2000, 4500, 4500, 4500, 4500, 12000, 300])

In [41]:
# 物理边界检查函数
def check(GeomData):
    inva_place1 = tf.where(tf.logical_or(GeomData < 0, GeomData > 1))
    GeomData = tf.tensor_scatter_nd_update(GeomData, [inva_place1], [np.random.uniform(size=(inva_place1.shape[0]))])
    
    return tf.Variable(GeomData)

In [42]:
freqs = np.reshape(np.arange(2.0, 7.1, 0.1), (-1, 1)).astype(np.float32)

In [51]:
d_lb = np.where(freqs == 3.7)[0][0]
d_rb = np.where(freqs == 5.3)[0][0]
o_l = np.where(freqs == 3.5)[0][0]
o_r = np.where(freqs == 5.5)[0][0]

In [52]:
# 初始数据
geom = tf.Variable(np.random.uniform(size=(num_nodes, 9)), dtype=tf.float32)

In [53]:
bestLoss = 10
bestGeom = 0

In [54]:
# 创建优化器
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)

In [55]:
logs = []

In [56]:
# 优化过程
t0 = time.time()
for i in tqdm(range(num_node_epochs)):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(geom)
        input = tf.concat([tf.tile(freqs, (num_nodes, 1)), tf.repeat(geom, freqs.shape[0], axis=0)], axis=1)
        y_pred = model(input, training=False)
        loss = dgn_obj(y_pred, 1, 10, 1, 1)
    minLoss = tf.reduce_min(loss).numpy()
    minIndex = tf.argmin(loss).numpy()
    minGeom = geom[minIndex].numpy()
    grads = tape.gradient(loss, geom)
    opt.apply_gradients(grads_and_vars=zip([grads], [geom]))
    geom = check(geom)
    if minLoss < bestLoss:
        bestLoss = minLoss
        bestGeom = minGeom
        print(minIndex)
        print(i, bestLoss)
        print(" ".join(map(str, bestGeom)))
        print()
    # logs.append({'epoch': i, 'time': time.time() - t0, 'acc': bestLoss})

  0%|                                                                                  | 2/1500 [00:00<02:08, 11.70it/s]

137
0 1.1071211
0.91837806 0.47341177 0.34816986 0.7425377 0.9930117 0.12850636 0.3745302 0.4583793 0.29748058

3228
1 1.0585264
0.79848564 0.8773313 0.1896671 0.095579654 0.60023856 0.08708886 0.33713055 0.59896135 0.50255084



  0%|▎                                                                                 | 6/1500 [00:00<01:56, 12.84it/s]

3228
3 1.0232123
0.8123149 0.8911605 0.19072041 0.09663295 0.59918565 0.088142134 0.33818385 0.6000147 0.5163804

2153
4 0.9849012
0.6351811 0.83898115 0.22789691 0.3109106 0.9273691 0.08842714 0.29787838 0.61299145 0.4551676

3228
5 0.984641
0.8119584 0.9024266 0.1910768 0.09698934 0.6104518 0.07687595 0.33854026 0.6003711 0.5276466



  1%|▍                                                                                 | 8/1500 [00:00<01:52, 13.21it/s]

...

 15%|███████████▉                                                                    | 224/1500 [00:16<01:35, 13.43it/s]

254
222 0.60633266
0.85579187 0.69041663 0.19792888 0.7139966 0.9950002 0.0108073475 0.45841646 0.44370574 0.66793257



 35%|████████████████████████████▎                                                   | 530/1500 [00:39<01:12, 13.45it/s]

2268
528 0.59797823
0.95926297 0.74431086 0.21496262 0.7175269 0.94296926 0.039020322 0.3673967 0.5048517 0.68799514



100%|███████████████████████████████████████████████████████████████████████████████| 1500/1500 [01:52<00:00, 13.35it/s]


In [59]:
output = bestGeom * (mmax - mmin) + mmin

In [60]:
print(' '.join(list(map(str, output))))

1926.6733407974243 1539.7595405578613 586.932709813118 3285.3657245635986 4254.767829179764 367.7873857319355 1779.805862903595 10514.555096626282 237.59902715682983


In [62]:
firstGemo = np.array([1263.710880279541, 1736.2465858459473, 359.1015040874481, 2136.1098557710648, 4175.255888700485, 200.68820547312498, 1907.0350915193558, 10947.9421377182, 252.18486785888672])

In [64]:
firstGemo = (firstGemo - mmin) / (mmax - mmin)

In [66]:
aa = model(tf.concat([freqs, tf.repeat([bestGeom], freqs.shape[0], axis=0)], axis=1)).numpy()
for i in range(aa.shape[0]):
    print(freqs[i][0], 20 * np.log10(np.sqrt(aa[i][0]**2 + aa[i][1]**2)))

2.0 -42.791315966917836
2.1 -41.33532547118391
2.2 -41.08394240297415
2.3 -42.05253337744119
2.4 -45.546476173406624
2.5 -50.5511838555485
2.6 -53.80682892334767
2.7 -47.3095635731669
2.8 -43.16976595123772
2.9 -40.40066369067528
3.0 -39.266773297893934
3.1 -41.60190613688735
3.2 -46.5373424796224
3.3 -28.82689621877052
3.4 -19.45302261742348
3.5 -7.049513289511008
3.6 -2.145352965847626
3.7 -1.9227014470477273
3.8 -0.8381757471606127
3.9 -0.9972341916206442
4.0 -0.38820690880626796
4.1 -0.982877168743397
4.2 -0.5308158657909657
4.3 -0.15523650551155643
4.4 -0.7151493593497498
4.5 -0.5866343534657835
4.6 -0.4216540377273941
4.7 -0.6076372200858298
4.8 -0.7911693546776897
4.9 -0.7968002081917966
5.0 -0.6821964584923468
5.1 -0.6901260459255355
5.2 -1.001137231350063
5.3 -0.9949511953632078
5.4 -3.458450161999046
5.5 -9.89260369387811
5.6 -17.039371860953295
5.7 -23.506378359275395
5.8 -29.159774951824744
5.9 -31.1018886830456
6.0 -34.50396829637487
6.1 -35.17042330104217
6.2 -36.55193195

In [74]:
20 * np.log10(0.805)

-1.8840823926426293

In [67]:
E = tf.reduce_sum(tf.square([aa]), axis=-1)

In [78]:
E[0][17]

<tf.Tensor: shape=(), dtype=float32, numpy=0.642288>

In [76]:
freqs[17]

array([3.7], dtype=float32)

In [71]:
for i in E[0]:
    print(i.numpy())

5.258579e-05
7.3530486e-05
7.791226e-05
6.233711e-05
2.7883827e-05
8.808087e-06
4.1621443e-06
1.8579913e-05
4.8197377e-05
9.118715e-05
0.00011839208
6.915274e-05
2.2195542e-05
0.0013101179
0.011342212
0.19726437
0.61018944
0.642288
0.82448435
0.79483426
0.91449076
0.79746616
0.8849493
0.9648868
0.8481742
0.87364817
0.9074749
0.8694333
0.83345675
0.8323768
0.8546344
0.8530754
0.79412025
0.7952522
0.45097762
0.102503724
0.019772556
0.0044602803
0.0012134517
0.00077590963
0.0003544893
0.00030405886
0.00022121104
0.00020122205
0.00018621612
0.00015683619
0.00012651547
0.00010590644
8.7764136e-05
9.96667e-05
7.217824e-05


In [21]:
freqs[lb + 1]

array([3.7], dtype=float32)

In [22]:
desired_E = E[: , lb + 1: rb]

In [23]:
desired_E

<tf.Tensor: shape=(1, 17), dtype=float32, numpy=
array([[0.65623575, 0.82333624, 0.79483694, 0.8844146 , 0.79324514,
        0.9119164 , 0.9655333 , 0.8584281 , 0.87947786, 0.8974844 ,
        0.88338107, 0.8409991 , 0.8259543 , 0.8866119 , 0.86257255,
        0.7958137 , 0.8109648 ]], dtype=float32)>

In [24]:
desired_P = 10**(-1 / 10)

In [25]:
desired_P

0.7943282347242815

In [26]:
desired_condition = desired_E < desired_P

In [27]:
a = desired_P**(1 - 3)

In [28]:
desired_E = tf.where(desired_condition, desired_P - desired_E, 0)

In [29]:
desired_E

<tf.Tensor: shape=(1, 17), dtype=float32, numpy=
array([[0.13809246, 0.        , 0.        , 0.        , 0.00108308,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]], dtype=float32)>

In [30]:
desired_E = E[: , lb: rb + 1]

In [31]:
desired_E

<tf.Tensor: shape=(1, 19), dtype=float32, numpy=
array([[0.61752844, 0.65623575, 0.82333624, 0.79483694, 0.8844146 ,
        0.79324514, 0.9119164 , 0.9655333 , 0.8584281 , 0.87947786,
        0.8974844 , 0.88338107, 0.8409991 , 0.8259543 , 0.8866119 ,
        0.86257255, 0.7958137 , 0.8109648 , 0.45140105]], dtype=float32)>

In [32]:


    

    
    

    desired = tf.reduce_sum(desired_E, axis=-1)